In [ ]:
import os
import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
%env SM_FRAMEWORK=tf.keras
import warnings
warnings.filterwarnings("ignore")
import os
import gc
import cv2
import json
import time
import random
import numpy as np
import pandas as pd
import tifffile as tiff
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import albumentations as albu
from sklearn.model_selection import train_test_split, KFold, GroupKFold
import tensorflow.keras.backend as K
from tensorflow.keras import Model, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import *
import segmentation_models as sm
from segmentation_models import Unet, FPN
from segmentation_models.losses import bce_jaccard_loss
print("tensorflow version:", tf.__version__)
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
if gpu_devices:
    for gpu_device in gpu_devices:
        print("device available:", gpu_device)
policy = tf.keras.mixed_precision.experimental.Policy("mixed_float16")
tf.keras.mixed_precision.experimental.set_policy(policy)
pd.set_option("display.max_columns", None)
%matplotlib inline

In [ ]:
!nvidia-smi -L

In [ ]:
VER = "v12"
PARAMS = {
    "version": VER,
    "folds": 2,
    "img_size": 256,
    "resize": 4,
    "batch_size" : 32,
    "epochs": 2,
    "patience": 20,
    "backbone": "efficientnetb0",# efficientnetbX, resnet34/50, resnext50, seresnet34, seresnext
    "loss": "bce_dice",
    "mirror": False,
    "umodel" : "unet",
    "bce_weight": 1.,
    "shift": True,
    "pseudo":"",
    "lr": 0.0002,
    "split": "kfold",
    "seed": None
}
DATA_PATH = "../input/hubmap-kidney-segmentation"


IMGS_PATH = f"{DATA_PATH}/tiles_r"
MSKS_PATH = f"{DATA_PATH}/masks_r"

MDLS_PATH = f"./models_{VER}"
if not os.path.exists(MDLS_PATH):
    os.mkdir(MDLS_PATH)
with open(f"{MDLS_PATH}/params.json", "w") as file:
    json.dump(PARAMS,file)
        

if not PARAMS["mirror"]:
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    STRATEGY = tf.distribute.get_strategy()
else:
    STRATEGY = tf.distribute.MirroredStrategy()
    
start_time = time.time()

In [ ]:
df_masks = pd.read_csv(f"{DATA_PATH}/train.csv").set_index("id")
df_masks

In [ ]:
def enc2mask(encs, shape):
    img = np.zeros(shape[0] * shape[1], dtype = np.uint8)
    for m, enc in enumerate(encs):
        if isinstance(enc, np.float) and np.isnan(enc): continue
        s = enc.split()
        for i in range(len(s) // 2):
            start = int(s[2 * i]) - 1
            length = int(s[2 * i + 1])
            img[start : start + length] = 1 + m
    return img.reshape(shape).T

def show_img_n_mask(df, img_num, resize):
    img = tiff.imread(os.path.join(f"{DATA_PATH}/train", df.index[img_num] + ".tiff"))
    if len(img.shape) == 5: img = np.transpose(img.squeeze(), (1, 2, 0))
    mask = enc2mask(df.iloc[img_num], (img.shape[1], img.shape[0]))
    print(img.shape, mask.shape)
    img = cv2.resize(img,
                     (img.shape[1] // resize, img.shape[0] // resize),
                     interpolation = cv2.INTER_AREA)
    mask = cv2.resize(mask,
                      (mask.shape[1] // resize, mask.shape[0] // resize),
                      interpolation = cv2.INTER_NEAREST)
    plt.figure(figsize = (8, 8))
    plt.axis("off")
    plt.imshow(img)
    plt.imshow(mask, alpha = 0.4)
    plt.show()

In [ ]:
class DataGenKid(Sequence):
    
    def __init__(self, imgs_path, msks_path, imgs_idxs, img_size, 
                 batch_size = 32, mode = "fit", shuffle = False,
                 resize = None):
        self.imgs_path = imgs_path
        self.msks_path = msks_path
        self.imgs_idxs = imgs_idxs
        self.img_size = img_size
        self.batch_size = batch_size
        self.mode = mode
        self.shuffle = shuffle
        self.resize = resize
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.imgs_idxs) / self.batch_size))
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.imgs_idxs))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def __getitem__(self, index):
        batch_size = min(self.batch_size, len(self.imgs_idxs) - index * self.batch_size)
        X = np.zeros((batch_size, self.img_size, self.img_size, 3), dtype = np.float32)
        imgs_batch = self.imgs_idxs[index * self.batch_size : (index + 1) * self.batch_size]
        if self.mode == "fit":
            y = np.zeros((batch_size, self.img_size, self.img_size), dtype = np.float32)
            for i, img_idx in enumerate(imgs_batch):
                X[i, ], y[i] = self.get_tile(img_idx)
            return X, y
        elif self.mode == "predict":
            for i, img_idx in enumerate(imgs_batch):
                X[i, ] = self.get_tile(img_idx)
            return X
        else:
            raise AttributeError("Fit mode parameter error")
            
    def get_tile(self, img_idx):
        img_path = f"{self.imgs_path}/{img_idx}.png"
        img = cv2.imread(img_path)
        if img is None:
            print(f"error loading image: {image_path}")
        img = img.astype(np.float32) / 255
        if self.mode == "fit":
            msk_path = f"{self.msks_path}/{img_idx}.png"
            msk = cv2.imread(msk_path, cv2.IMREAD_GRAYSCALE)
            if msk is None:
                print(f"error loading mask: {msk_path}")
            msk = msk.astype(np.float32)
            return img, msk
        else:
            return img
        

In [ ]:
imgs_idxs = [x.replace(".png", "") for x in os.listdir(IMGS_PATH) if ".png" in x]
train_datagen = DataGenKid(
        imgs_path = IMGS_PATH,
        msks_path = MSKS_PATH,
        imgs_idxs = imgs_idxs,
        img_size = PARAMS["img_size"],
        batch_size = PARAMS["batch_size"],
        mode = "fit",
        shuffle = True,
        resize = None
)

val_datagen = DataGenKid(
        imgs_path = IMGS_PATH,
        msks_path = MSKS_PATH,
        imgs_idxs = imgs_idxs,
        img_size = PARAMS["img_size"],
        batch_size = PARAMS["batch_size"],
        mode = "fit",
        shuffle = False,
        resize = None
)

In [ ]:
def plot_item(item_nr):
    bsize = min(8, PARAMS["batch_size"])
    Xt, yt = train_datagen.__getitem__(item_nr)
    print("test X: ", Xt.shape)
    print("test y: ", yt.shape)
    fig, axes = plt.subplots(figsize = (16, 4), nrows = 2, ncols = bsize)
    for j in range(bsize):
        axes[0, j].imshow(Xt[j])
        axes[0, j].set_title(j)
        axes[0, j].axis("off")
        axes[1, j].imshow(yt[j])
        axes[1, j].axis("off")
    plt.show()

In [ ]:
plot_item(223)

In [ ]:
def dice_coef(y_true, y_pred, smooth = 1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2 * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred, smooth = 1):
    return (1 - dice_coef(y_true, y_pred, smooth))

def bce_dice_loss(y_true, y_pred):
    return PARAMS["bce_weight"] * binary_crossentropy(y_true, y_pred) + \
        (1 - PARAMS["bce_weight"]) * dice_loss(y_true, y_pred)

def get_model(backbone, input_shape, loss_type = "bce_dice",
              umodel = "unet", classes = 1, lr = .001):
    with STRATEGY.scope():
        if loss_type == "bce_dice":
            loss = bce_dice_loss
        
        # TODO: Implement other bce_jaccard_loss
            
        else: 
            raise AttributeError("loss mode parameter error")
        
        if umodel == "unet":
            model = Unet(backbone_name = backbone, encoder_weights = "imagenet",
                         input_shape = input_shape,
                         classes = classes, activation = "sigmoid")
            
        elif umodel == "fpn":
            model = FPN(backbone_name = backbone, encoder_weights = "imagenet",
                        input_shape = input_shape,
                        classes = classes, activation = "sigmoid")
        
        else:
            raise AttributeError("umodel mode parameter error")
            
        model.compile(
            optimizer = tfa.optimizers.Lookahead(
                tf.keras.optimizers.Adam(learning_rate = lr),
                sync_period = max(6, int(PARAMS["patience"] / 4))
            ),
            loss = loss,
            metrics = [dice_coef]
        )
    return model

## Train

In [ ]:
def train_model(mparams, n_fold, train_datagen, val_datagen):
    model = get_model(
        mparams["backbone"],
        input_shape = (mparams["img_size"], mparams["img_size"], 3),
        loss_type = mparams["loss"],
        umodel = mparams["umodel"],
        lr = mparams["lr"]
    )
    checkpoint_path = f"{MDLS_PATH}/model_{n_fold}.hdf5"
    earlystopper = EarlyStopping (
        monitor = "val_dice_coef",
        patience = mparams["patience"],
        verbose = 0,
        restore_best_weights = True,
        mode = "max"
    )
    lrreducer = ReduceLROnPlateau(
        monitor = "val_dice_coef",
        factor = .1,
        patience = int(mparams["patience"] / 2),
        verbose = 0,
        min_lr = 1e-7,
        mode = "max"
    )
    checkpointer = ModelCheckpoint(
        checkpoint_path,
        monitor = "val_dice_coef",
        verbose = 0,
        save_best_only = True,
        save_weights_only = True,
        mode = "max"
    )
    callbacks = [earlystopper, checkpointer]
    callbacks.append(lrreducer)
    print("lr reduce on plateau")
    history = model.fit(
        train_datagen,
        validation_data = val_datagen,
        callbacks = callbacks,
        epochs = mparams["epochs"],
        verbose = 1
    )
    history_file = f"{MDLS_PATH}/history_{n_fold}.json"
    dict_to_save = {}
    for k, v in history.history.items():
        dict_to_save.update({k: [np.format_float_positional(x) for x in history.history[k]]})
    with open(history_file, "w") as file:
        json.dump(dict_to_save, file)
    model.load_weights(checkpoint_path)
    return model, history  

In [ ]:
for iname in list(set([x[:9] for x in imgs_idxs])):
    print("img name:", iname,
          "| imgs number:", len([x for x in imgs_idxs if x[:9] == iname]))
if PARAMS["split"] == "kfold":
    kfold = KFold(n_splits = PARAMS["folds"],
                  random_state = PARAMS["seed"],
                  shuffle = True).split(imgs_idxs)
else:
    raise AttributeError("split mode parameter error")

In [ ]:
epoch_by_folds = []
loss_by_folds = []
dice_coef_by_folds = []

for n, (tr, te) in enumerate(kfold):
    print("=" * 10, f"FOLD {n}", "=" * 10)
    X_tr = [imgs_idxs[i] for i in tr]; X_val = [imgs_idxs[i] for i in te]
    print("train:", len(X_tr), "| test:", len(X_val))
    print("groups train:", set([x[:9] for x in X_tr]),
          "\ngroups test:", set([x[:9] for x in X_val]))
    train_datagen = DataGenKid(
        imgs_path = IMGS_PATH,
        msks_path = MSKS_PATH,
        imgs_idxs = X_tr,
        img_size = PARAMS["img_size"],
        batch_size = PARAMS["batch_size"],
        mode = "fit",
        shuffle = True,
        resize = None
    )
    
    val_datagen = DataGenKid(
        imgs_path = IMGS_PATH,
        msks_path = MSKS_PATH,
        imgs_idxs = X_val,
        img_size = PARAMS["img_size"],
        batch_size = PARAMS["batch_size"],
        mode = "fit",
        shuffle = False,
        resize = None
    )
    model, history = train_model(PARAMS, n, train_datagen, val_datagen)
    
    plt.plot(history.history["loss"], label = "loss")
    plt.plot(history.history["val_loss"], label = "val_loss")
    plt.legend()
    plt.show()
    plt.plot(history.history["dice_coef"], label = "dice_coef")
    plt.plot(history.history["val_dice_coef"], label = "val_dice_coef")
    plt.legend()
    plt.show()
    best_epoch = np.argmax(history.history["val_dice_coef"])
    best_loss = history.history["val_loss"][best_epoch]
    best_dice_coef = history.history["val_dice_coef"][best_epoch]
    print("best epoch:", best_epoch,
         "| best loss:", best_loss,
         "| best dice coef:", best_dice_coef)
    epoch_by_folds.append(best_epoch)
    loss_by_folds.append(best_loss)
    dice_coef_by_folds.append(best_dice_coef)
    del train_datagen, val_datagen, model; gc.collect()

elapsed_time = time.time() - start_time
print(f"time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec")

In [ ]:
result = PARAMS.copy()
result["bavg_epoch"] = np.mean(epoch_by_folds)
result["bavg_loss"] = np.mean(loss_by_folds)
result["bavg_dice_coef"] = np.mean(dice_coef_by_folds)
result["dice_by_folds"] = " ".join([f"{x:.4f}" for x in dice_coef_by_folds])
with open(f"{MDLS_PATH}/params.json", "w") as file:
    json.dump(result, file)
if not os.path.exists("results.csv"):
    df_save = pd.DataFrame(result, index = [0])
    df_save.to_csv("results.csv", sep = "\t")
else:
    df_old = pd.read_csv("results.csv", sep = "\t", index_col = 0)
    df_save = pd.DataFrame(result, index = [df_old.index.max() + 1])
    df_save = df_old.append(df_save, ignore_index = True)
    df_save.to_csv("results.csv", sep = "\t")

In [ ]:
pd.read_csv("results.csv", sep = "\t", index_col=0).iloc[:,4:]

## Predict

In [ ]:
larger = 4
test_models = []
for n_fold in list(range(PARAMS["folds"])):
    
    checkpoint_path = f"{MDLS_PATH}/model_{n_fold}.hdf5"
    print(checkpoint_path)
    model_lrg = get_model (
        PARAMS["backbone"],
        input_shape = (PARAMS["img_size"] * larger, PARAMS["img_size"] * larger, 3),
        loss_type = PARAMS["loss"],
        umodel = PARAMS["umodel"]
    )
    model_lrg.load_weights(checkpoint_path) # or .set_weights(model.get_weights()) from smaller model
    test_models.append(model_lrg)

In [ ]:
img_num = 0
resize = PARAMS["resize"]
shft = 0.6
wnd = PARAMS["img_size"] * larger
img = tiff.imread(os.path.join(f"{DATA_PATH}/train", df_masks.index[img_num] + ".tiff"))
if len(img.shape) == 5: img = np.transpose(img.squeeze(), (1, 2, 0))
mask = enc2mask(df_masks.iloc[img_num], (img.shape[1], img.shape[0]))
print(img.shape, mask.shape)
img = cv2.resize(img,
                 (img.shape[1] // resize, img.shape[0] // resize),
                 interpolation = cv2.INTER_AREA)
mask = cv2.resize(mask,
                  (mask.shape[1] // resize, mask.shape[0] // resize),
                  interpolation = cv2.INTER_NEAREST)
img = img[int(img.shape[0] * shft) : int(img.shape[0] * shft) + wnd,
          int(img.shape[1] * shft) : int(img.shape[1] * shft) + wnd,
          :]
mask = mask[int(mask.shape[0] * shft) : int(mask.shape[0] * shft) + wnd,
            int(mask.shape[1] * shft) : int(mask.shape[1] * shft) + wnd]
plt.figure(figsize = (4, 4))
plt.axis("off")
plt.imshow(img)
print(img.shape)
plt.imshow(mask, alpha = 0.4)
plt.show()

In [ ]:
def dice_np(pred, true, k = 1):
    intersection = np.sum(pred[true == k]) * 2
    dice = intersection / (np.sum(pred) + np.sum(true))
    return dice

def get_dice(mask, mask_lrg, th):
    mask_pred = np.squeeze(mask_lrg > th).astype(int)
    return dice_np(mask, mask_pred)

def get_best_th_dice(mask, mask_lrg, n=100, plot = False):
    thresholds = np.linspace(0, 1, n)
    dices = [get_dice(mask, mask_lrg, th) for th in thresholds]
    n_max = np.argmax(dices)
    if plot:
        plt.plot(thresholds, dices)
        plt.title(f"th: {thresholds[n_max]:.2f} dice: {dices[n_max]:.2f}")
        plt.show()
    return thresholds[n_max], dices[n_max]

In [ ]:
fig, axes = plt.subplots(figsize = (16, 4), nrows = 1, ncols = len(test_models))
for j in range(len(test_models)):
    mask_lrg = test_models[j].predict(img[np.newaxis, ] / 255)
    axes[j].imshow(np.squeeze(mask_lrg))
    axes[j].set_title(f"img {j}: {np.min(mask_lrg):.2f}-{np.max(mask_lrg):.2f}")
    axes[j].axis("off")
    print(get_best_th_dice(mask, mask_lrg))
plt.show()

In [ ]:
get_best_th_dice(mask, mask_lrg, n = 100, plot = True)

In [ ]:
plt.figure(figsize = (14, 4))
plt.hist(mask_lrg.flatten(), bins = 100)
plt.show()

In [ ]:
plt.figure(figsize = (14, 4))
plt.hist(np.where(mask_lrg < 10e-4, np.nan, mask_lrg).flatten(), bins = 100)
plt.show()